In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195808 sha256=cdcd9485fc9d21bae2e98b08b4cd235c7e33675281ed74b30f8fad4b42c790d2
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [4]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 49.8 MB/s eta 0:00:00


In [5]:
from surprise import Dataset, Reader
from surprise import SVD, SVDpp, BaselineOnly

from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [6]:
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.lmf import LogisticMatrixFactorization

/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

In [7]:
random_state_value = 42

rating_column_name = 'rating'

**Загружаем датасет**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
data_dir = '/content/drive/MyDrive/хакатон_работа_ру'

In [10]:
train_df = pd.read_parquet(f"{data_dir}/Processed_dataset.parquet")
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type
0,1,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
2,1,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
3,1,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy
4,1,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy
...,...,...,...,...,...,...,...,...
12292583,367641,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292584,367641,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts
12292585,367641,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy
12292586,367641,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy


In [11]:
test_df = pd.read_parquet(f"{data_dir}/test_public_mfti.parquet")
test_df  

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [12]:
nonrel_act_list = ['show_vacancy',
                'preview_click_vacancy',
                'click_favorite',
                'preview_click_favorite']

rel_act_list = ['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

In [13]:
def get_top_n_vacancies(rating_act_list, pos_vacancies_start=0, pos_vacancies_end=100):
    show_act = train_df[train_df['event_type'].isin(rating_act_list)].groupby('vacancy_id_')['event_type'].count()
    top_n_with_event_quant = show_act.sort_values(ascending=False).iloc[pos_vacancies_start:pos_vacancies_end].to_frame().reset_index()
    return top_n_with_event_quant['vacancy_id_'].unique()

In [14]:
vac_start = 0
vac_end = 1000
rating_act = ['show_vacancy']

top1000vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top1000vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251, 164588, 154411, 155539, 120252, 230707, 127352,
       214513, 182084, 249571, 207156, 168935, 239021, 184440, 106944,
       187360, 206350, 117525, 111592, 150283, 253678, 112506, 117532,
       220718, 151616, 109079, 111837, 136266, 164246, 182439, 111890,
       257631, 143721, 248852, 171332, 128183, 140917, 111941, 212141,
       102794, 212325, 229689, 258378, 106293, 176171, 129787, 181275,
       190928, 242560, 195702, 191512, 144113, 209568, 219681, 197930,
       239624, 128278, 230322, 221108, 171602, 154423, 205606, 209422,
      

In [49]:
# ТОП-1000 для рейтинга по просмотрам

vac_start = 0
vac_end = 1000
rating_act = ['show_vacancy']

top1000vac = get_top_n_vacancies(rating_act,
                               pos_vacancies_start=vac_start,
                               pos_vacancies_end=vac_end)
top1000vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251, 164588, 154411, 155539, 120252, 230707, 127352,
       214513, 182084, 249571, 207156, 168935, 239021, 184440, 106944,
       187360, 206350, 117525, 111592, 150283, 253678, 112506, 117532,
       220718, 151616, 109079, 111837, 136266, 164246, 182439, 111890,
       257631, 143721, 248852, 171332, 128183, 140917, 111941, 212141,
       102794, 212325, 229689, 258378, 106293, 176171, 129787, 181275,
       190928, 242560, 195702, 191512, 144113, 209568, 219681, 197930,
       239624, 128278, 230322, 221108, 171602, 154423, 205606, 209422,
      

In [15]:
# ТОП-80 для рейтинга по просмотрам

vac_start = 0
vac_end = 80
rating_act = ['show_vacancy']

top80vac = get_top_n_vacancies(rating_act,
                               pos_vacancies_start=vac_start,
                               pos_vacancies_end=vac_end)
top80vac

array([260154, 198114, 203404, 202608, 116823, 164602, 207423, 148714,
       111505, 182870, 250327, 111867, 110421, 108242, 242642, 217683,
       258441, 158242, 162187, 174953, 176141, 207108, 247535, 110792,
       237341, 113305, 240744, 149024, 244077, 113482, 180382, 114583,
       182100, 113707, 105907, 210628, 227708, 110793, 247276, 181745,
       164481, 193331, 246509, 243868, 190030, 153245, 138634, 169194,
       115924, 126251, 164588, 154411, 155539, 120252, 230707, 127352,
       214513, 182084, 249571, 207156, 168935, 239021, 184440, 106944,
       187360, 206350, 117525, 111592, 150283, 253678, 112506, 117532,
       220718, 151616, 109079, 111837, 136266, 164246, 182439, 111890])

In [48]:
# все вакансии:
allvac = train_df.groupby('vacancy_id_') \
           .cookie_id.nunique().sort_values(ascending = False).shape

In [57]:
# ТОП-25000 для рейтинга по просмотрам

vac_start = 0
vac_end = 25000
rating_act = ['show_vacancy']

top25000vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)


In [70]:
# ТОП-70000 для рейтинга по просмотрам

vac_start = 0
vac_end = 70000
rating_act = ['show_vacancy']

top70000vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top70000vac

array([260154, 198114, 203404, ..., 178808, 192985, 213121])

In [69]:
# ТОП-100000 для рейтинга по просмотрам

vac_start = 0
vac_end = 100000
rating_act = ['show_vacancy']

top100000vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top100000vac

array([260154, 198114, 203404, ..., 150150, 129688, 118359])

In [96]:
# Топ вакансий с 3 до 95

vac_start = 3
vac_end = 95
rating_act = ['show_vacancy']

top95vac_from3 = get_top_n_vacancies(rating_act,
                                                      pos_vacancies_start=vac_start,
                                                      pos_vacancies_end=vac_end)
top95vac_from3

array([202608, 116823, 164602, 207423, 148714, 111505, 182870, 250327,
       111867, 110421, 108242, 242642, 217683, 258441, 158242, 162187,
       174953, 176141, 207108, 247535, 110792, 237341, 113305, 240744,
       149024, 244077, 113482, 180382, 114583, 182100, 113707, 105907,
       210628, 227708, 110793, 247276, 181745, 164481, 193331, 246509,
       243868, 190030, 153245, 138634, 169194, 115924, 126251, 164588,
       154411, 155539, 120252, 230707, 127352, 214513, 182084, 249571,
       207156, 168935, 239021, 184440, 106944, 187360, 206350, 117525,
       111592, 150283, 253678, 112506, 117532, 220718, 151616, 109079,
       111837, 136266, 164246, 182439, 111890, 257631, 143721, 248852,
       171332, 128183, 140917, 111941, 212141, 102794, 212325, 229689,
       258378, 106293, 176171, 129787])

In [68]:
# топ 25000 вакансий по релевантным действиям
rel_act_ = train_df.groupby('vacancy_id_')['rating'].count()
topvac = rel_act_.sort_values(ascending=False).iloc[0:25000].to_frame().reset_index()
top25000rel =topvac['vacancy_id_'].unique()
top25000rel

array([260154, 198114, 203404, ..., 129435, 249061, 132159])

In [73]:
# топ 50000 вакансий по релевантным действиям
rel_act_ = train_df.groupby('vacancy_id_')['rating'].count()
topvac = rel_act_.sort_values(ascending=False).iloc[0:50000].to_frame().reset_index()
top50000rel =topvac['vacancy_id_'].unique()
top50000rel

array([260154, 198114, 203404, ..., 167622, 247663, 212423])

In [76]:
# топ 70000 вакансий по релевантным действиям
rel_act_ = train_df.groupby('vacancy_id_')['rating'].count()
topvac = rel_act_.sort_values(ascending=False).iloc[0:70000].to_frame().reset_index()
top70000rel =topvac['vacancy_id_'].unique()
top70000rel

array([260154, 198114, 203404, ..., 163190, 210308, 197805])

In [62]:
# топ 100000 вакансий по релевантным действиям
rel_act_ = train_df.groupby('vacancy_id_')['rating'].count()
topvac = rel_act_.sort_values(ascending=False).iloc[0:100000].to_frame().reset_index()
top100000rel =topvac['vacancy_id_'].unique()
top100000rel

array([260154, 198114, 203404, ..., 130251, 228713, 228874])

In [18]:
all_act_last = train_df.groupby('vacancy_id_')['rating'].count()
topvac = all_act_last.sort_values(ascending=False).iloc[3:95].to_frame().reset_index()
top_last =topvac['vacancy_id_'].unique()
topvac

,vacancy_id_,rating
0,202608,41620
1,116823,40023
2,207423,38607
3,164602,36850
4,148714,35629
...,...,...
87,129787,5966
88,258378,5964
89,190928,5949
90,209568,5926


In [17]:
# Создаем колонку с типом действия для рейтинга
# 1 - целевое действие, 0 - остальные действия

def check_is_purpose(event):
    if event in rel_act_list:
        return 1
    else:
        return 0

train_df[rating_column_name] = train_df['event_type'].apply(check_is_purpose)
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,rating
0,1,2022-08-30,1661866548,0.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
1,1,2022-08-30,1661866548,11.0,153975,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
2,1,2022-08-30,1661866559,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
3,1,2022-08-30,1661866559,5.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,preview_click_vacancy,0
4,1,2022-08-30,1661866564,0.0,182445,0000c4548c3944c08972bbdc1fa4eb85,2eb30fab80244cdebbafd5ed096bc08f,show_vacancy,0
...,...,...,...,...,...,...,...,...,...
12292583,367641,2022-08-22,1661164755,0.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292584,367641,2022-08-22,1661164755,791.0,160164,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_contacts,1
12292585,367641,2022-08-22,1661165546,1.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,preview_click_vacancy,0
12292586,367641,2022-08-22,1661165547,5.0,185412,ffffdb17f45b4032b386d691d52e6c00,None,show_vacancy,0


In [19]:
# Готовим датафрейм для обучения модели:

train_matrix = train_df.groupby(['cookie_id', 'vacancy_id_'], as_index=False)[rating_column_name].sum()
train_matrix

,cookie_id,vacancy_id_,rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0
...,...,...,...
4678580,ffffdb17f45b4032b386d691d52e6c00,107125,1
4678581,ffffdb17f45b4032b386d691d52e6c00,107893,1
4678582,ffffdb17f45b4032b386d691d52e6c00,150843,1
4678583,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [20]:
# Функция для создания рейтинга на основе порогового значения для целевых действий

def create_rating(input_rating, threshold = 1):
    rating = input_rating
    if rating > threshold:
        rating = 1
    return rating

In [21]:
# Создаем рейтинг
# Принципы построения
# 1. rating >= 1 - rating = 1
# 2. rating = 0 - оставляем без изменения

rating_threshold = 1

train_matrix_0_1 = train_matrix.copy()
train_matrix_0_1[rating_column_name] = train_matrix_0_1[rating_column_name].apply(create_rating, threshold = rating_threshold)
train_matrix_0_1
train_matrix_0_1[rating_column_name].unique()

array([0, 1])

In [22]:
train_matrix_0_1

,cookie_id,vacancy_id_,rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0
...,...,...,...
4678580,ffffdb17f45b4032b386d691d52e6c00,107125,1
4678581,ffffdb17f45b4032b386d691d52e6c00,107893,1
4678582,ffffdb17f45b4032b386d691d52e6c00,150843,1
4678583,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [99]:
median_vacancies_per_cookie = train_matrix_0_1.groupby('cookie_id')['vacancy_id_'].nunique().median()
mean_vacancies_per_cookie = train_matrix_0_1.groupby('cookie_id')['vacancy_id_'].nunique().mean()

9.0

In [72]:
cookie_ids_to_exclude = train_matrix_0_1.groupby('cookie_id').filter(lambda x: (x['vacancy_id_'].nunique() > 8) and (x[x['rating'] == 1]['vacancy_id_'].nunique() >= 5))['cookie_id'].unique()

KeyboardInterrupt: ignored

In [71]:
cookie_ids_to_exclude.shape

(17184,)

In [73]:
# Выбрать только те cookie_id, у которых количество вакансий больше 8 (1000 cлуйчаных) 
import random
random.seed(13)
# Выбрать cookie_id с количеством vacancy_id_ > 8 и рейтингом == 1
cookie_ids_to_exclude = train_matrix_0_1.groupby('cookie_id').filter(lambda x: (x['vacancy_id_'].nunique() > 8) and (x[x['rating'] == 1]['vacancy_id_'].nunique() >= 5))['cookie_id'].unique()

# Выбрать случайные 1000 уникальных cookie_id
cookie_ids_to_exclude = random.sample(list(cookie_ids_to_exclude), 1000)

# Выбрать только те cookie_id, которые нужно исключить
train_5_exclude = train_matrix_0_1[train_matrix_0_1['cookie_id'].isin(cookie_ids_to_exclude)]

# Получить последние 5 vacancy_id_ с рейтингом == 1 для каждого cookie_id исключаемых из train_5
test_5 = train_5_exclude.groupby('cookie_id').apply(lambda x: x[x['rating'] == 1].tail(5)).reset_index(drop=True)





In [75]:
merged = train_matrix_0_1.merge(test_5[['cookie_id', 'vacancy_id_']], how='left', on=['cookie_id', 'vacancy_id_'], indicator=True)
train_5 = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [76]:
train_5

,cookie_id,vacancy_id_,rating
0,0000c4548c3944c08972bbdc1fa4eb85,137659,0
1,0000c4548c3944c08972bbdc1fa4eb85,153975,0
2,0000c4548c3944c08972bbdc1fa4eb85,174953,0
3,0000c4548c3944c08972bbdc1fa4eb85,176171,0
4,0000c4548c3944c08972bbdc1fa4eb85,182445,0
...,...,...,...
4678580,ffffdb17f45b4032b386d691d52e6c00,107125,1
4678581,ffffdb17f45b4032b386d691d52e6c00,107893,1
4678582,ffffdb17f45b4032b386d691d52e6c00,150843,1
4678583,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [60]:
train_matrix_0_1.query("cookie_id == 'ff79149b6a7045cfa0fd04d5b7281349'")

,cookie_id,vacancy_id_,rating
4668973,ff79149b6a7045cfa0fd04d5b7281349,106208,1
4668974,ff79149b6a7045cfa0fd04d5b7281349,111867,0
4668975,ff79149b6a7045cfa0fd04d5b7281349,172390,0
4668976,ff79149b6a7045cfa0fd04d5b7281349,179082,0
4668977,ff79149b6a7045cfa0fd04d5b7281349,182084,0
4668978,ff79149b6a7045cfa0fd04d5b7281349,188439,0
4668979,ff79149b6a7045cfa0fd04d5b7281349,196428,1
4668980,ff79149b6a7045cfa0fd04d5b7281349,198751,0
4668981,ff79149b6a7045cfa0fd04d5b7281349,203404,0
4668982,ff79149b6a7045cfa0fd04d5b7281349,207423,0


In [77]:
test_5 = test_5.rename(columns={'removed_vacancies':'vacancy_id_'})


In [78]:
test_5

,cookie_id,vacancy_id_,rating
0,002d5aa1ac26466f914e87983c47cb0d,126489,1
1,002d5aa1ac26466f914e87983c47cb0d,134469,1
2,002d5aa1ac26466f914e87983c47cb0d,155179,1
3,002d5aa1ac26466f914e87983c47cb0d,158242,1
4,002d5aa1ac26466f914e87983c47cb0d,165580,1
...,...,...,...
4995,ff939426a68d43d88a222da586d45818,228224,1
4996,ff939426a68d43d88a222da586d45818,242661,1
4997,ff939426a68d43d88a222da586d45818,242662,1
4998,ff939426a68d43d88a222da586d45818,244770,1


In [79]:
test_6 = test_5.groupby(['cookie_id'], as_index=False).agg({'vacancy_id_': list})
test_6

,cookie_id,vacancy_id_
0,002d5aa1ac26466f914e87983c47cb0d,"[126489, 134469, 155179, 158242, 165580]"
1,003b1f1319b640e4b856670d84e75e9c,"[123467, 124440, 128054, 153062, 176817]"
2,0052234b5356480f9dff207888135548,"[185567, 189054, 203017, 205881, 245864]"
3,00734ab7548749a688c99fbd2274fb92,"[221357, 223608, 225701, 230074, 249609]"
4,00c475b6abbb402bbb2baced30dc236e,"[149024, 169194, 182870, 212325, 255782]"
...,...,...
995,fdfc683b118a4b92befa63b196748761,"[222294, 222381, 229672, 230280, 237341]"
996,fe1ca58dc20e41fb89a7bf65e68da4bb,"[137317, 154386, 163726, 165105, 203769]"
997,fe4dbc900fbd49f689b2b78acc5d4b4f,"[145786, 158697, 159072, 160164, 255357]"
998,fefd4bb67bb24e3fbecb668c414e904a,"[131647, 165098, 182879, 212234, 249325]"


In [41]:
# Готовим данные для обучения модели

reader = Reader(rating_scale=(0, 1))
train_data_0_1 = Dataset.load_from_df(train_5[['cookie_id', 'vacancy_id_', rating_column_name]], reader)

#reader = Reader(rating_scale=(0, 2))
#train_data_0_2 = Dataset.load_from_df(train_matrix_0_2[['cookie_id', 'vacancy_id_', rating_column_name]], reader)

In [42]:
# создаем объект surprise.dataset.Dataset для обучающего и тестового датасета
svd_20epochs_0_1 = SVD(verbose=True, n_epochs=20)
trainset_0_1 = train_data_0_1.build_full_trainset()
svd_20epochs_0_1.fit(trainset_0_1)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [43]:
col_names ={
    'id_col_name': 'cookie_id',
    'vac_col_name': 'vacancy_id_',
    'rating_col_name': rating_column_name
}

In [44]:
def transform_into_sparse_matrix(input_df, col_names):
    input_df_fixed = input_df.copy()
    input_df_fixed[col_names['id_col_name']] = input_df_fixed[col_names['id_col_name']].astype('category')
    input_df_fixed[col_names['vac_col_name']] = input_df_fixed[col_names['vac_col_name']].astype('category')
    input_df_fixed[col_names['rating_col_name']] = input_df_fixed[col_names['rating_col_name']].astype('uint8')
    user_item_matrix = csr_matrix((input_df_fixed[col_names['rating_col_name']], 
                                (input_df_fixed[col_names['id_col_name']].cat.codes, 
                                 input_df_fixed[col_names['vac_col_name']].cat.codes)))
    id_cat_codes = dict(zip(input_df_fixed[col_names['id_col_name']], input_df_fixed[col_names['id_col_name']].cat.codes))
    vac_cat_codes = dict(zip(input_df_fixed[col_names['vac_col_name']], input_df_fixed[col_names['vac_col_name']].cat.codes))
    cat_vac_codes = dict(zip(input_df_fixed[col_names['vac_col_name']].cat.codes, input_df_fixed[col_names['vac_col_name']]))
    return (user_item_matrix, id_cat_codes, vac_cat_codes, cat_vac_codes)


In [45]:
Impl_0_1_set = transform_into_sparse_matrix(train_5, col_names)

In [46]:
Impl_ALS_0_1 = AlternatingLeastSquares(factors=100,
                                       regularization=0.05,
                                       calculate_training_loss=True,
                                       random_state=random_state_value) # , alpha=3.0
Impl_ALS_0_1.fit(Impl_0_1_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
Impl_LMF_0_1 = LogisticMatrixFactorization(factors=50, learning_rate=2, random_state=42)
Impl_LMF_0_1.fit(user_item_0_1[0])

  0%|          | 0/30 [00:00<?, ?it/s]

In [31]:
Impl_0_2_set = transform_into_sparse_matrix(train_matrix_0_2, col_names)

In [32]:
Impl_ALS_0_2 = AlternatingLeastSquares(factors=100,
                                       regularization=0.05,
                                       calculate_training_loss=True,
                                       random_state=random_state_value) # , alpha=3.0
Impl_ALS_0_2.fit(Impl_0_2_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
Impl_LMF_0_2 = LogisticMatrixFactorization(factors=50, learning_rate=2, random_state=42)
Impl_LMF_0_2.fit(Impl_0_2_set[0])

  0%|          | 0/30 [00:00<?, ?it/s]

In [56]:
# Вспомогательные функции

def get_predictions(cookie_id, model_info, initial_set, n_predict = 5):
    # Получим все вакансии с которыми взаимодействовал cookie_id:
    interacted_items = train_matrix[train_matrix['cookie_id'] == cookie_id]['vacancy_id_'].unique()
    
    # Удалим из этого перечня вакансии с которыми контактировал юзер:
    items_to_predict = np.setdiff1d(initial_set, interacted_items)
    
    # Создадим тест датасет для данного юзера:
    test_set = [[cookie_id, item, 4.] for item in items_to_predict]    
    
    model = model_info[0]
    model_type = model_info[1]  
    
    # получим топ-n вакансий для данного юзера:
    if model_type == 'surprise':
        top_n_vacancies = get_predictions_surprise(cookie_id, model, test_set, n_predict)
    
    if model_type == 'implicit':
        top_n_vacancies = get_predictions_implicit(cookie_id, model, items_to_predict, model_info[2], n_predict)
    
    
    return top_n_vacancies

def get_predictions_surprise(cookie_id, model, test_set, n_predict = 5):
    predictions = model.test(test_set)
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_predict]
    return [pred.iid for pred in top_n]

def get_predictions_implicit(cookie_id, model, test_set, impl_set, n_predict = 5):
    user_item = impl_set[0]
    id_cat_code = impl_set[1][cookie_id]
    vac_cat_codes = impl_set[2]
    cat_vac_codes = impl_set[3]
    filter_items = np.array([vac_cat_codes[i] for i in test_set])
    ids, scores = model.recommend(id_cat_code,
                                  user_item[id_cat_code],
                                  N=n_predict,
                                  filter_items=filter_items,
                                  filter_already_liked_items=True)
    return [cat_vac_codes[i] for i in ids]    

def calc_correct_recomend_number(predict_col, vac_col):
    """Функция для определения количества совпадающих вакансий в двух списках"""
    return len(set(predict_col) & set(vac_col))


def get_prec_n(res_df, num_corr_column, n_predict = 5):
    """Функция для расчета precision@n метрики"""
    corr_recommend = res_df[num_corr_column].sum()
    prec_5 = corr_recommend / (n_predict * res_df.shape[0])
    return {'num_corr_recomend': corr_recommend, f'precision@{n_predict}': prec_5}

In [63]:
# Копируем датасет чтобы не менять изначальный

test_df_with_pred = test_6.copy()


In [64]:
# Задаем словари с моделями и датасетами чтобы перебрать их все

models_dict = {
    
    'Impl_ALS_rat01': [Impl_ALS_0_1, 'implicit', Impl_0_1_set],
    
    # 'SVD_20ep_rat01': [svd_20epochs_0_1, 'surprise']
  
}

set_dict = {
     
       
    'top100000rel' : top100000rel
   # 'top100000vac_show' : top100000vac
    
}

In [54]:
# Импортируем tqdm для progress bar
from tqdm.auto import tqdm

def get_res_df(models_dict, set_dict, n_predict = 5):
    res_dict = {}
    for model_name in tqdm(models_dict):        
        for set_name in tqdm(set_dict):        
            # Создаем столбец с предсказаниями
            predict_col_name = f'predict_id_{model_name}_{set_name}_n_{n_pred}'            
            test_df_with_pred[predict_col_name] = test_df_with_pred['cookie_id'].apply(get_predictions,
                                                                                       model_info = models_dict[model_name],
                                                                                       initial_set = set_dict[set_name],
                                                                                       n_predict = n_pred)                  
            
            # Создаем столбец с количеством правильных предсказаний для каждого id
            num_corr_col_name = f'num_corr_{model_name}_{set_name}_n_{n_pred}'        
            test_df_with_pred[num_corr_col_name] = test_df_with_pred.apply(
                lambda x: calc_correct_recomend_number(x[predict_col_name], x['vacancy_id_']), axis=1
            )
            
            # Считаем precision@n метрику
            res_dict[model_name+'_'+set_name] = get_prec_n(test_df_with_pred,
                                                           num_corr_column = num_corr_col_name,
                                                           n_predict = n_pred)
    return pd.DataFrame.from_dict(res_dict, orient='index')

In [65]:
n_pred = 5

res_df = get_res_df(models_dict, set_dict, n_predict = n_pred)
res_df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,num_corr_recomend,precision@5
Impl_ALS_rat01_top100000rel,985,0.197


In [61]:
test_df_with_pred


,cookie_id,vacancy_id_,predict_id_Impl_ALS_rat01_top1000vac_n_5,num_corr_Impl_ALS_rat01_top1000vac_n_5,predict_id_SVD_20ep_rat01_top1000vac_n_5,num_corr_SVD_20ep_rat01_top1000vac_n_5
0,0035c298d8c64f368ae730a9cca9bb20,"[167837, 184047, 193078, 206957, 221527]","[106282, 245088, 257004, 126121, 176509]",0,"[180382, 182870, 178415, 101396, 174953]",0
1,004854d31ada47aea967913717c88981,"[104328, 114975, 160547, 170084, 258433]","[230136, 230142, 223140, 231733, 230425]",0,"[101396, 211897, 178415, 245952, 174953]",0
2,005943cd85264bf0a667fa7037b2c90b,"[162187, 178143, 185913, 187888, 202608]","[177188, 172944, 145549, 177694, 167001]",0,"[182870, 174953, 178415, 180382, 221920]",0
3,00789706a7bd4ec79138f8c6926ddf77,"[212676, 230840, 235614, 245213, 258663]","[235614, 162169, 207764, 191664, 179095]",1,"[180382, 219948, 221920, 182870, 230145]",0
4,00c15cb6e3c84464a0e538e46ced6c46,"[251534, 253886, 254058, 256132, 259050]","[256132, 259050, 145344, 119095, 176777]",2,"[210229, 178415, 182870, 174953, 180382]",0
...,...,...,...,...,...,...
995,ff0cd5f894ac458bb558896fa1c1f9c4,"[109186, 111958, 120952, 181305, 244319]","[132116, 164067, 226719, 141555, 139347]",0,"[182870, 178415, 101396, 221920, 174953]",0
996,ff29db0512864555a1c21c504a04fa35,"[141621, 150199, 175337, 248051, 251088]","[163067, 257004, 132116, 217765, 154498]",0,"[163625, 180382, 174953, 219948, 101396]",0
997,ff3d9eee97a44c2db37b0ab1f443b196,"[198114, 202608, 219681, 245637, 256058]","[245637, 198114, 167001, 211501, 104345]",2,"[221920, 210229, 174953, 180382, 220718]",0
998,ff5b9f2242bb475fae24821cb6002991,"[102794, 217683, 250327, 255596]","[230136, 230142, 230425, 231733, 223140]",0,"[221920, 219948, 182870, 101396, 235677]",0


KeyError: ignored